HDP-base spell-sound analyzer
developed by Kow Kuroda (kow.kuroda@gmail.com)

History
2024/08/19 Cython を使った cy_gen_ngrams を利用可能にした

In [81]:
#!pip install -U pyLDAvis
#!pip install -U pandas

In [82]:
## imports
import os, sys
import pprint as pp
import unicodedata

In [83]:
## 一つ上の階層のファイルを見るように設定
sys.path.append(os.path.join(os.path.dirname("__file__"), '..'))

In [84]:
## Cython を使うかどうか
use_Cython = False
##
if use_Cython:
    %load_ext Cython

The Cython extension is already loaded. To reload it, use:
  %reload_ext Cython


In [85]:
## Cython module の生成 (必要に応じて)
#!python clean setup.py build_ext --inplace

In [86]:
## term settings
term_classes        = [ 'spell', 'sound' ]
term_class          = term_classes[0]
ngram_is_inclusive  = True
## doc settings
max_doc_size        = 11
min_doc_size        =  3
print(f"max_doc_size: {max_doc_size}")
print(f"min_doc_size: {min_doc_size}")
### boundary handling
add_boundary       = False
boundary_mark      = "#"
## term setting
seg_joint          = ""
gap_mark           = "…"
term_is_skippy     = True
n_for_ngram        = 4
max_gap_ratio      = 0.8
max_gap_size       = round(max_doc_size * max_gap_ratio)
print(f"term_class: {term_class}")
print(f"term_is_skippy: {term_is_skippy}")
print(f"max_gap_size: {max_gap_size}")
print(f"n_for_ngram: {n_for_ngram}")
### accent handling
suppress_accents   = True
accent_marks       = [ "ˈ", "ˌ" ] 
if term_class == 'sound':
    if suppress_accents:
        accent_status = "-unaccented"
    else:
        accent_stratus = "-accented"
else:
    accent_status = ""
print(f"accent_status: {accent_status}")
## define term_type
if term_class == 'spell':
    if term_is_skippy:
        term_type = f"sp_skippy{n_for_ngram}gram"
    else:
        term_type = f"sp_{n_for_ngram}gram"
else:
    if term_is_skippy:
        term_type = f"sn_skippy{n_for_ngram}gram"
    else:
        term_type = f"sn_{n_for_ngram}gram"
## check
print(f"term_type: {term_type}")

max_doc_size: 11
min_doc_size: 3
term_class: spell
term_is_skippy: True
max_gap_size: 9
n_for_ngram: 4
accent_status: 
term_type: sp_skippy4gram


In [87]:
## target language
## a key must be part of a file name 
target_lang_dict = {    'en_US' : 'English (US)',
                        'en_UK' : 'English (UK)',
                        'en_N_only' : 'English noun (WN)',
                        'en_V_only' : 'English verb (WN)',
                        'en_A_only' : 'English adj (WN)',
                        'en_R_only' : 'English adv (WN)',
                        'ar'    : 'Arabic',
                        'de'    : 'German',
                        'de_N_only' : 'German Nouns',
                        'de_non_N_only' : 'German Non-nouns',
                        'eo'    : 'Esperanto',
                        'es_ES' : 'Spanish (Spain)',
                        'es_MX' : 'Spanish (Mexico)',
                        'fi'    : 'Finnish',
                        'fr_FR' : 'French (France)',
                        'fr_QC' : 'French (Quebec)',
                        'is'    : 'Icelandic',
                        'ir'    : 'Irish',
                        'nl'    : 'Dutch',
                        'ro'    : 'Romanian',
                        'sw'    : 'Swahili' }
## proper language selection
target_lang_keys = [    'en_US', # 0
                        'en_UK', # 1
                        'en_N_only', # 2
                        'en_V_only', # 3
                        'en_A_only', # 4
                        'en_R_only', # 5
                        'ar', # 6
                        'de', # 7
                        'de_N_only', # 8
                        'de_non_N_only', # 9
                        'eo', 'es_ES', 'es_MX',
                        'fi', 'fr_FR', 'fr_QC',
                        'is', 'nl', 'ro', 'sw',
                        'ir' # This lacks sound
                    ]
## check
target_lang_key  = target_lang_keys[4]
print(f"target_lang_key: {target_lang_key}")
print(f"target lang: {target_lang_dict[target_lang_key]} [{target_lang_key}]")
## target_attr [effective only for Irish]
target_class = ""
#target_class = None # This causes an unrediable error
if target_lang_key == "ir":
    target_classes = [ 'adjectives', 'nouns', 'verbs' ]
    target_class = f"-{target_classes[3]}"
print(f"target_class: {target_class}")

target_lang_key: en_A_only
target lang: English adj (WN) [en_A_only]
target_class: 


In [88]:
## LDA/HDP
apply_term_filtering = True
## The following parameters need to be relatively large to prevent "Row sum not equal 1" error
term_minfreq       = 2
## The following value is crucial to prevent "Row sum not equal 1" error
abuse_threshold    = 0.04 # larger value selects shorter units, smaller value selects longer units
min_bot_size       = 3
# number of terms listed for a given topic
n_terms_to_show    = 120

In [89]:
## sampling
source_sampling          = True
source_sampling_rate     = 0.5
source_sampling_max_size = 5000
second_sampling          = False
second_sampling_rate     = 0.7

In [90]:
## set target files
import glob
data_dir1     = "data/open-dict-ipa/data1"
data_dir2     = "data/open-dict-ipa/data1a"
data_dir3     = "data/wn3"
data_dir4     = "data/irish"
target_files  = glob.glob(f"{data_dir1}/*")
target_files2 = glob.glob(f"{data_dir2}/*")
target_files.extend(target_files2)
target_files3 = glob.glob(f"{data_dir3}/*")
target_files.extend(target_files3)
target_files4 = glob.glob(f"{data_dir4}/*")
target_files.extend(target_files4)
#
target_files = sorted([ file for file in target_files if ".csv" in file ])
pp.pprint(target_files)

['data/irish/word-irish-adjectives-spell.csv',
 'data/irish/word-irish-noun-phrases-spell.csv',
 'data/irish/word-irish-nouns-spell.csv',
 'data/irish/word-irish-possessives-spell.csv',
 'data/irish/word-irish-prepositions-spell.csv',
 'data/irish/word-irish-verbs-spell.csv',
 'data/open-dict-ipa/data1/ar.csv.gz',
 'data/open-dict-ipa/data1/de.csv.gz',
 'data/open-dict-ipa/data1/en_UK.csv.gz',
 'data/open-dict-ipa/data1/en_US.csv.gz',
 'data/open-dict-ipa/data1/eo.csv.gz',
 'data/open-dict-ipa/data1/es_ES.csv.gz',
 'data/open-dict-ipa/data1/es_MX.csv.gz',
 'data/open-dict-ipa/data1/fa.csv.gz',
 'data/open-dict-ipa/data1/fi.csv.gz',
 'data/open-dict-ipa/data1/fr_FR.csv.gz',
 'data/open-dict-ipa/data1/fr_QC.csv.gz',
 'data/open-dict-ipa/data1/is.csv.gz',
 'data/open-dict-ipa/data1/ja.csv.gz',
 'data/open-dict-ipa/data1/jam.csv.gz',
 'data/open-dict-ipa/data1/ma.csv.gz',
 'data/open-dict-ipa/data1/nb.csv.gz',
 'data/open-dict-ipa/data1/nl.csv.gz',
 'data/open-dict-ipa/data1/or.csv.gz',
 '

In [91]:
## get source data from files
import pandas as pd
import gzip
#target_language_key = "en_US" # can be changed to get other languages
#if target_class != "" or target_class is not None:
if target_class != "":
    target_file = [ f for f in target_files if target_lang_key in f and target_class in f ][0]
else:
    target_file = [ f for f in target_files if target_lang_key in f ][0]
print(f"processing: {target_file}")
##
if target_lang_key == "ir":
    col_names = ['spell', 'POS']
else:
    col_names = ['spell', 'sound']
#
if target_file.endswith(".gz"):
    with gzip.open(target_file, "rt") as f:
        raw_df = pd.read_csv(f, encoding = 'utf8', header = None, names = col_names )
else:
    with open(target_file, "rt") as f:
        raw_df = pd.read_csv(f, encoding = 'utf8', header = None, names = col_names )
## normalize characters
raw_df['spell'] = raw_df['spell'].apply(lambda x: unicodedata.normalize('NFC', str(x)))
## modify sound
try:
    sounds = raw_df['sound'].apply(lambda x: x.strip('/') )
    sounds = [ x.split("/,")[0] for x in sounds ] # picks up only the first of multiple entries
    raw_df['sound'] = sounds
except (AttributeError, KeyError):
    pass
#
raw_df.sample(10)

processing: data/wn3/en_A_only.csv


,spell,sound
1274,conversant,kənˈvɝsənt
5492,unimportant,ənɪmˈpɔɹtənt
4425,republican,ɹiˈpəbɫəkən
4284,racking,ˈɹækɪŋ
1378,cryogenic,ˈkɹaɪəˌdʒɛnɪk
1851,eighty,ˈeɪti
5305,twenty,ˈtwɛni
1879,elusive,ɪˈɫusɪv
1226,conspicuous,kənˈspɪkjuəs
5440,undisciplined,ənˈdɪsɪpɫɪnd


In [92]:
## source sampling
len(raw_df)
if source_sampling:
	print(f"source sampling applied")
	if len(raw_df) >= source_sampling_max_size:
		raw_df = raw_df.sample(source_sampling_max_size)
	else:
		raw_df = raw_df.sample(round(len(raw_df) * source_sampling_rate))
## remove accent marking
if suppress_accents:
	try:
		raw_df['sound'] = raw_df['sound'].apply(lambda x: "".join([ y for y in list(x) if y not in accent_marks ]))
	except KeyError:
		pass
## add boudary marks
if add_boundary:
	raw_df['spell'] = raw_df['spell'].apply(lambda x: f"{boundary_mark}{x}{boundary_mark}")
	try:
		raw_df['sound'] = raw_df['sound'].apply(lambda x: f"{boundary_mark}{x}{boundary_mark}")
	except KeyError:
		pass
#
print(raw_df)

source sampling applied
           spell      sound
5669      unwary     ənwɛɹi
4267       quasi      kwɑsi
353   appointive   əpɔɪntɪv
742       bottom      bɑtəm
1750        drab       dɹæb
...          ...        ...
5101    tangible  tændʒəbəɫ
2623      hushed       həʃt
4714       showy       ʃoʊi
3173      lipped       ɫɪpt
3639    nontoxic  nɑntɑksɪk

[5000 rows x 2 columns]


In [93]:
## generate 1-grams for spell and sound
## spell
raw_df['sp_1gram'] = raw_df['spell'].apply(lambda x: list(str(x)))
# add column of size
raw_df['sp_size'] = raw_df['sp_1gram'].apply(lambda x: len(x))
# add column of count of '-' inside
raw_df['hyphen'] = raw_df['sp_1gram'].apply(lambda x: list(x).count("-"))
# add column of count of '.' inside
raw_df['period'] = raw_df['sp_1gram'].apply(lambda x: list(x).count("."))
## sound
# takes the first entry, removes '/' around
try:
    raw_df['sn_1gram'] = raw_df['sound'].apply(lambda x: list(x) )
except (TypeError, KeyError):
    pass
# add column of size
try:
    raw_df['sn_size'] = raw_df['sn_1gram'].apply(lambda x: len(x))
except KeyError:
    pass
## check
raw_df

,spell,sound,sp_1gram,sp_size,hyphen,period,sn_1gram,sn_size
5669,unwary,ənwɛɹi,"[u, n, w, a, r, y]",6,0,0,"[ə, n, w, ɛ, ɹ, i]",6
4267,quasi,kwɑsi,"[q, u, a, s, i]",5,0,0,"[k, w, ɑ, s, i]",5
353,appointive,əpɔɪntɪv,"[a, p, p, o, i, n, t, i, v, e]",10,0,0,"[ə, p, ɔ, ɪ, n, t, ɪ, v]",8
742,bottom,bɑtəm,"[b, o, t, t, o, m]",6,0,0,"[b, ɑ, t, ə, m]",5
1750,drab,dɹæb,"[d, r, a, b]",4,0,0,"[d, ɹ, æ, b]",4
...,...,...,...,...,...,...,...,...
5101,tangible,tændʒəbəɫ,"[t, a, n, g, i, b, l, e]",8,0,0,"[t, æ, n, d, ʒ, ə, b, ə, ɫ]",9
2623,hushed,həʃt,"[h, u, s, h, e, d]",6,0,0,"[h, ə, ʃ, t]",4
4714,showy,ʃoʊi,"[s, h, o, w, y]",5,0,0,"[ʃ, o, ʊ, i]",4
3173,lipped,ɫɪpt,"[l, i, p, p, e, d]",6,0,0,"[ɫ, ɪ, p, t]",4


In [94]:
## filtering raw_data by size
print(f"term_type: {term_type}")
if "sp_" in term_type:
    df_filtered = raw_df[ (raw_df['sp_size'] <= max_doc_size) & (raw_df['sp_size'] >= min_doc_size) & (raw_df['hyphen'] == 0) & (raw_df['period'] == 0) ]
else:
    df_filtered = raw_df[ (raw_df['sn_size'] <= max_doc_size) & (raw_df['sn_size'] >= min_doc_size) ]
#
df_filtered

term_type: sp_skippy4gram


,spell,sound,sp_1gram,sp_size,hyphen,period,sn_1gram,sn_size
5669,unwary,ənwɛɹi,"[u, n, w, a, r, y]",6,0,0,"[ə, n, w, ɛ, ɹ, i]",6
4267,quasi,kwɑsi,"[q, u, a, s, i]",5,0,0,"[k, w, ɑ, s, i]",5
353,appointive,əpɔɪntɪv,"[a, p, p, o, i, n, t, i, v, e]",10,0,0,"[ə, p, ɔ, ɪ, n, t, ɪ, v]",8
742,bottom,bɑtəm,"[b, o, t, t, o, m]",6,0,0,"[b, ɑ, t, ə, m]",5
1750,drab,dɹæb,"[d, r, a, b]",4,0,0,"[d, ɹ, æ, b]",4
...,...,...,...,...,...,...,...,...
5101,tangible,tændʒəbəɫ,"[t, a, n, g, i, b, l, e]",8,0,0,"[t, æ, n, d, ʒ, ə, b, ə, ɫ]",9
2623,hushed,həʃt,"[h, u, s, h, e, d]",6,0,0,"[h, ə, ʃ, t]",4
4714,showy,ʃoʊi,"[s, h, o, w, y]",5,0,0,"[ʃ, o, ʊ, i]",4
3173,lipped,ɫɪpt,"[l, i, p, p, e, d]",6,0,0,"[ɫ, ɪ, p, t]",4


In [95]:
## define df after second sampling if any
len(df_filtered)
if second_sampling:
    df = df_filtered.sample(round(len(df_filtered) * second_sampling_rate))
else:
    df = df_filtered
len(df)

4454

In [96]:
## spell 2grams
#import ngrams
import gen_ngrams
module_name = "gen_ngrams"
reload_module = False
if reload_module:
    import importlib
    importlib.reload(module_name)

if term_class == 'spell':
    #sp_2grams = [ ngrams.list_gen_ngrams (x, n = 2, check = False) for x in df['sp_1gram'] ]
    sp_2grams = [ gen_ngrams.gen_ngrams (x, n = 2, sep = "", check = False) for x in df['sp_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sp_2grams):
            g.extend(list(df['sp_1gram'])[i])
    ## add sp_2gram
    df['sp_2gram'] = sp_2grams

/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_44770/3511996102.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sp_2gram'] = sp_2grams


In [97]:
## spell 3grams
#import ngrams
import gen_ngrams
if n_for_ngram > 2 and term_class == 'spell':
    #sp_3grams = [ ngrams.list_gen_ngrams (x, n = 3, check = False) for x in df['sp_1gram'] ]
    sp_3grams = [ gen_ngrams.gen_ngrams (x, n = 3, sep = "", check = False) for x in df['sp_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sp_3grams):
            g.extend(list(df['sp_2gram'])[i])
    ## add sp_3gram
    df['sp_3gram'] = sp_3grams

/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_44770/1405656247.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sp_3gram'] = sp_3grams


In [98]:
## spell 4grams
#import ngrams
import gen_ngrams
if n_for_ngram > 3 and term_class == 'spell':
    #sp_4grams = [ ngrams.list_gen_ngrams (x, n = 4, check = False) for x in df['sp_1gram'] ]
    sp_4grams = [ gen_ngrams.gen_ngrams (x, n = 4, sep = "", check = False) for x in df['sp_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sp_4grams):
            g.extend(list(df['sp_3gram'])[i])
    ## add sp_4gram
    df['sp_4gram'] = sp_4grams

/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_44770/1571879696.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sp_4gram'] = sp_4grams


In [99]:
## spell 5grams
#import ngrams
import gen_ngrams
if n_for_ngram > 4 and term_class == 'spell':
    sp_5grams = [ gen_ngrams.gen_ngrams (x, n = 5, sep = "", check = False) for x in df['sp_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sp_5grams):
            g.extend(list(df['sp_4gram'])[i])
    ## add sp_5gram
    df['sp_5gram'] = sp_5grams

In [100]:
df

,spell,sound,sp_1gram,sp_size,hyphen,period,sn_1gram,sn_size,sp_2gram,sp_3gram,sp_4gram
5669,unwary,ənwɛɹi,"[u, n, w, a, r, y]",6,0,0,"[ə, n, w, ɛ, ɹ, i]",6,"[un, nw, wa, ar, ry, u, n, w, a, r, y]","[unw, nwa, war, ary, un, nw, wa, ar, ry, u, n,...","[unwa, nwar, wary, unw, nwa, war, ary, un, nw,..."
4267,quasi,kwɑsi,"[q, u, a, s, i]",5,0,0,"[k, w, ɑ, s, i]",5,"[qu, ua, as, si, q, u, a, s, i]","[qua, uas, asi, qu, ua, as, si, q, u, a, s, i]","[quas, uasi, qua, uas, asi, qu, ua, as, si, q,..."
353,appointive,əpɔɪntɪv,"[a, p, p, o, i, n, t, i, v, e]",10,0,0,"[ə, p, ɔ, ɪ, n, t, ɪ, v]",8,"[ap, pp, po, oi, in, nt, ti, iv, ve, a, p, p, ...","[app, ppo, poi, oin, int, nti, tiv, ive, ap, p...","[appo, ppoi, poin, oint, inti, ntiv, tive, app..."
742,bottom,bɑtəm,"[b, o, t, t, o, m]",6,0,0,"[b, ɑ, t, ə, m]",5,"[bo, ot, tt, to, om, b, o, t, t, o, m]","[bot, ott, tto, tom, bo, ot, tt, to, om, b, o,...","[bott, otto, ttom, bot, ott, tto, tom, bo, ot,..."
1750,drab,dɹæb,"[d, r, a, b]",4,0,0,"[d, ɹ, æ, b]",4,"[dr, ra, ab, d, r, a, b]","[dra, rab, dr, ra, ab, d, r, a, b]","[drab, dra, rab, dr, ra, ab, d, r, a, b]"
...,...,...,...,...,...,...,...,...,...,...,...
5101,tangible,tændʒəbəɫ,"[t, a, n, g, i, b, l, e]",8,0,0,"[t, æ, n, d, ʒ, ə, b, ə, ɫ]",9,"[ta, an, ng, gi, ib, bl, le, t, a, n, g, i, b,...","[tan, ang, ngi, gib, ibl, ble, ta, an, ng, gi,...","[tang, angi, ngib, gibl, ible, tan, ang, ngi, ..."
2623,hushed,həʃt,"[h, u, s, h, e, d]",6,0,0,"[h, ə, ʃ, t]",4,"[hu, us, sh, he, ed, h, u, s, h, e, d]","[hus, ush, she, hed, hu, us, sh, he, ed, h, u,...","[hush, ushe, shed, hus, ush, she, hed, hu, us,..."
4714,showy,ʃoʊi,"[s, h, o, w, y]",5,0,0,"[ʃ, o, ʊ, i]",4,"[sh, ho, ow, wy, s, h, o, w, y]","[sho, how, owy, sh, ho, ow, wy, s, h, o, w, y]","[show, howy, sho, how, owy, sh, ho, ow, wy, s,..."
3173,lipped,ɫɪpt,"[l, i, p, p, e, d]",6,0,0,"[ɫ, ɪ, p, t]",4,"[li, ip, pp, pe, ed, l, i, p, p, e, d]","[lip, ipp, ppe, ped, li, ip, pp, pe, ed, l, i,...","[lipp, ippe, pped, lip, ipp, ppe, ped, li, ip,..."


In [101]:
## spell skippy 2gram
if term_class == 'spell':
    if use_Cython:
        import cy_gen_ngrams
        sp_skippy2grams = [ [seg_joint.join(x) for x in cy_gen_ngrams.cy_gen_skippy_ngrams(x, n = 2, check = False)] for x in list(df['sp_1gram']) ]

    else:
        import gen_ngrams
        sp_skippy2grams = [ gen_ngrams.gen_skippy_ngrams(x, 2, sep = "", max_distance = max_gap_size, missing_mark = gap_mark, check = False) for x in df['sp_1gram'] ]
        if ngram_is_inclusive:
            for i, g in enumerate(sp_skippy2grams):
                g.extend(list(df['sp_1gram'])[i])
##
df['sp_skippy2gram'] = sp_skippy2grams
df['sp_skippy2gram']

/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_44770/1187622597.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sp_skippy2gram'] = sp_skippy2grams


5669    [unwa, unw…r, unw…y, un…ar, un…a…y, un…ry, u…w...
4267                    [quas, qua…i, qu…si, q…asi, uasi]
353     [appo, app…i, app…n, app…t, app…i, app…v, app…...
742     [bott, bot…o, bot…m, bo…to, bo…t…m, bo…om, b…t...
1750                                               [drab]
                              ...                        
5101    [tang, tan…i, tan…b, tan…l, tan…e, ta…gi, ta…g...
2623    [hush, hus…e, hus…d, hu…he, hu…h…d, hu…ed, h…s...
4714                    [show, sho…y, sh…wy, s…owy, howy]
3173    [lipp, lip…e, lip…d, li…pe, li…p…d, li…ed, l…p...
3639    [nont, non…o, non…x, non…i, non…c, no…to, no…t...
Name: sp_skippy2gram, Length: 4454, dtype: object

In [102]:
term_class

'spell'

In [103]:
## spell skippy 3gram
#import ngrams_skippy
import gen_ngrams
if n_for_ngram > 2 and term_class == 'spell':
    #sp_skippy3grams = [ ngrams_skippy.gen_skippy3grams(x, missing_mark = gap_mark, check = False) for x in df['sp_1gram'] ]
    ## The code above was replaced by the following more efficient one
    sp_skippy3grams = [ gen_ngrams.gen_skippy_ngrams(x, 3, sep = "", max_distance = max_gap_size, missing_mark = gap_mark, check = False) for x in df['sp_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sp_skippy3grams):
            g.extend(list(df['sp_skippy2gram'])[i])
    #
    df['sp_skippy3gram'] = sp_skippy3grams
    df['sp_skippy3gram']

/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_44770/3806328476.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sp_skippy3gram'] = sp_skippy3grams


In [104]:
## spell skippy 4gram
#import ngrams_skippy
import gen_ngrams
if n_for_ngram > 3 and term_class == 'spell':
    #sp_skippy4grams = [ ngrams_skippy.gen_skippy4grams(x, missing_mark = gap_mark, check = False) for x in df['sp_1gram'] ]
    ## The code above was replaced by the following more efficient one
    sp_skippy4grams = [ gen_ngrams.gen_skippy_ngrams(x, 4, sep = "", max_distance = max_gap_size, missing_mark = gap_mark, check = False) for x in df['sp_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sp_skippy4grams):
            g.extend(list(df['sp_skippy3gram'])[i])
    #
    df['sp_skippy4gram'] = sp_skippy4grams

/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_44770/1288956926.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['sp_skippy4gram'] = sp_skippy4grams


In [105]:
## spell skippy 5gram
import gen_ngrams
if n_for_ngram > 4 and term_class == 'spell':
    sp_skippy5grams = [ gen_ngrams.gen_skippy_ngrams(x, 5, sep = "", max_distance = max_gap_size, missing_mark = gap_mark, check = False) for x in df['sp_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sp_skippy5grams):
            g.extend(list(df['sp_skippy4gram'])[i])
    #
    df['sp_skippy5gram'] = sp_skippy5grams

In [106]:
df

,spell,sound,sp_1gram,sp_size,hyphen,period,sn_1gram,sn_size,sp_2gram,sp_3gram,sp_4gram,sp_skippy2gram,sp_skippy3gram,sp_skippy4gram
5669,unwary,ənwɛɹi,"[u, n, w, a, r, y]",6,0,0,"[ə, n, w, ɛ, ɹ, i]",6,"[un, nw, wa, ar, ry, u, n, w, a, r, y]","[unw, nwa, war, ary, un, nw, wa, ar, ry, u, n,...","[unwa, nwar, wary, unw, nwa, war, ary, un, nw,...","[unwa, unw…r, unw…y, un…ar, un…a…y, un…ry, u…w...","[unw, un…a, un…r, un…y, u…wa, u…w…r, u…w…y, u…...","[unwa, unw…r, unw…y, un…ar, un…a…y, un…ry, u…w..."
4267,quasi,kwɑsi,"[q, u, a, s, i]",5,0,0,"[k, w, ɑ, s, i]",5,"[qu, ua, as, si, q, u, a, s, i]","[qua, uas, asi, qu, ua, as, si, q, u, a, s, i]","[quas, uasi, qua, uas, asi, qu, ua, as, si, q,...","[quas, qua…i, qu…si, q…asi, uasi]","[qua, qu…s, qu…i, q…as, q…a…i, q…si, uas, ua…i...","[quas, qua…i, qu…si, q…asi, uasi, uasi, qua, q..."
353,appointive,əpɔɪntɪv,"[a, p, p, o, i, n, t, i, v, e]",10,0,0,"[ə, p, ɔ, ɪ, n, t, ɪ, v]",8,"[ap, pp, po, oi, in, nt, ti, iv, ve, a, p, p, ...","[app, ppo, poi, oin, int, nti, tiv, ive, ap, p...","[appo, ppoi, poin, oint, inti, ntiv, tive, app...","[appo, app…i, app…n, app…t, app…i, app…v, app…...","[app, ap…o, ap…i, ap…n, ap…t, ap…i, ap…v, a…po...","[appo, app…i, app…n, app…t, app…i, app…v, ap…o..."
742,bottom,bɑtəm,"[b, o, t, t, o, m]",6,0,0,"[b, ɑ, t, ə, m]",5,"[bo, ot, tt, to, om, b, o, t, t, o, m]","[bot, ott, tto, tom, bo, ot, tt, to, om, b, o,...","[bott, otto, ttom, bot, ott, tto, tom, bo, ot,...","[bott, bot…o, bot…m, bo…to, bo…t…m, bo…om, b…t...","[bot, bo…t, bo…o, bo…m, b…tt, b…t…o, b…t…m, b…...","[bott, bot…o, bot…m, bo…to, bo…t…m, bo…om, b…t..."
1750,drab,dɹæb,"[d, r, a, b]",4,0,0,"[d, ɹ, æ, b]",4,"[dr, ra, ab, d, r, a, b]","[dra, rab, dr, ra, ab, d, r, a, b]","[drab, dra, rab, dr, ra, ab, d, r, a, b]",[drab],"[dra, dr…b, d…ab, rab, rab, drab]","[drab, dra, dr…b, d…ab, rab, rab, drab]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5101,tangible,tændʒəbəɫ,"[t, a, n, g, i, b, l, e]",8,0,0,"[t, æ, n, d, ʒ, ə, b, ə, ɫ]",9,"[ta, an, ng, gi, ib, bl, le, t, a, n, g, i, b,...","[tan, ang, ngi, gib, ibl, ble, ta, an, ng, gi,...","[tang, angi, ngib, gibl, ible, tan, ang, ngi, ...","[tang, tan…i, tan…b, tan…l, tan…e, ta…gi, ta…g...","[tan, ta…g, ta…i, ta…b, ta…l, ta…e, t…ng, t…n…...","[tang, tan…i, tan…b, tan…l, tan…e, ta…gi, ta…g..."
2623,hushed,həʃt,"[h, u, s, h, e, d]",6,0,0,"[h, ə, ʃ, t]",4,"[hu, us, sh, he, ed, h, u, s, h, e, d]","[hus, ush, she, hed, hu, us, sh, he, ed, h, u,...","[hush, ushe, shed, hus, ush, she, hed, hu, us,...","[hush, hus…e, hus…d, hu…he, hu…h…d, hu…ed, h…s...","[hus, hu…h, hu…e, hu…d, h…sh, h…s…e, h…s…d, h…...","[hush, hus…e, hus…d, hu…he, hu…h…d, hu…ed, h…s..."
4714,showy,ʃoʊi,"[s, h, o, w, y]",5,0,0,"[ʃ, o, ʊ, i]",4,"[sh, ho, ow, wy, s, h, o, w, y]","[sho, how, owy, sh, ho, ow, wy, s, h, o, w, y]","[show, howy, sho, how, owy, sh, ho, ow, wy, s,...","[show, sho…y, sh…wy, s…owy, howy]","[sho, sh…w, sh…y, s…ow, s…o…y, s…wy, how, ho…y...","[show, sho…y, sh…wy, s…owy, howy, howy, sho, s..."
3173,lipped,ɫɪpt,"[l, i, p, p, e, d]",6,0,0,"[ɫ, ɪ, p, t]",4,"[li, ip, pp, pe, ed, l, i, p, p, e, d]","[lip, ipp, ppe, ped, li, ip, pp, pe, ed, l, i,...","[lipp, ippe, pped, lip, ipp, ppe, ped, li, ip,...","[lipp, lip…e, lip…d, li…pe, li…p…d, li…ed, l…p...","[lip, li…p, li…e, li…d, l…pp, l…p…e, l…p…d, l…...","[lipp, lip…e, lip…d, li…pe, li…p…d, li…ed, l…p..."


In [107]:
## sound 2grams
import gen_ngrams
module_name = "gen_ngrams"
reload_module = False
if reload_module:
    import importlib
    importlib.reload(module_name)
#
if term_class == 'sound':
    sn_2grams = [ gen_ngrams.gen_ngrams(x, n = 2, sep ="", check = False) for x in df['sn_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sn_2grams):
            g.extend(list(df['sn_1gram'])[i])
    ## add sn_2gram
    df['sn_2gram'] = sn_2grams

In [108]:
## sound 3grams
import gen_ngrams
if n_for_ngram > 2 and term_class == 'sound':
    sn_3grams = [ gen_ngrams.gen_ngrams(x, n = 3, sep = "", check = False) for x in df['sn_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sn_3grams):
            g.extend(list(df['sn_2gram'])[i])
    ## add sn_3gram
    df['sn_3gram'] = sn_3grams

In [109]:
## sound 4grams
import gen_ngrams
if n_for_ngram > 3 and term_class == 'sound':
    sn_4grams = [ gen_ngrams.gen_ngrams (x, n = 4, sep = "", check = False) for x in df['sn_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sn_4grams):
            g.extend(list(df['sn_3gram'])[i])
    ## add sn_4gram
    df['sn_4gram'] = sn_4grams

In [110]:
## sound 5grams
import gen_ngrams
if n_for_ngram > 4 and term_class == 'sound':
    sn_5grams = [ gen_ngrams.gen_ngrams (x, n = 5, sep = "", check = False) for x in df['sn_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sn_5grams):
            g.extend(list(df['sn_4gram'])[i])
    ## add sn_4gram
    df['sn_5gram'] = sn_5grams

In [111]:
## sound skippy 2gram
#import ngrams_skippy
import gen_ngrams
if term_class == 'sound':
    sn_skippy2grams = [ gen_ngrams.gen_skippy_ngrams(x, n = 2, sep = "", max_distance = max_gap_size, missing_mark = gap_mark, check = False) for x in df['sn_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sn_skippy2grams):
            g.extend(list(df['sn_1gram'])[i])
    #
    df['sn_skippy2gram'] = sn_skippy2grams

In [112]:
## sound skippy 3gram
import gen_ngrams
if n_for_ngram > 2 and term_class == 'sound':
    sn_skippy3grams = [ gen_ngrams.gen_skippy_ngrams(x, n = 3, sep = "", max_distance = max_gap_size, missing_mark = gap_mark, check = False) for x in df['sn_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sn_skippy3grams):
            g.extend(list(df['sn_skippy2gram'])[i])
    #
    df['sn_skippy3gram'] = sn_skippy3grams

In [113]:
## sound skippy 4gram
#import ngrams_skippy
import gen_ngrams
if n_for_ngram > 3 and term_class == 'sound':
    sn_skippy4grams = [ gen_ngrams.gen_skippy_ngrams(x, n = 4, sep = "", max_distance = max_gap_size, missing_mark = gap_mark, check = False) for x in df['sn_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sn_skippy4grams):
            g.extend(list(df['sn_skippy3gram'])[i])
    #
    df['sn_skippy4gram'] = sn_skippy4grams

In [114]:
## sound skippy 5gram
import gen_ngrams
if n_for_ngram > 4 and term_class == 'sound':
    sn_skippy5grams = [ gen_ngrams.gen_skippy_ngrams(x, n = 5, sep = "", max_distance = max_gap_size, missing_mark = gap_mark, check = False) for x in df['sn_1gram'] ]
    if ngram_is_inclusive:
        for i, g in enumerate(sn_skippy5grams):
            g.extend(list(df['sn_skippy4gram'])[i])
    #
    df['sn_skippy5gram'] = sn_skippy5grams

In [115]:
## check df
dropped_vars = [ 'sp_size', 'hyphen', 'period', 'sn_size' ]
if term_class == 'spell':
    extra = [ 'sn_1gram', 'sn_2gram', 'sn_3gram', 'sn_4gram',
             'sn_skippy2gram', 'sn_skippy3gram', 'sn_skippy4gram' ]
    dropped_vars.extend(extra)
    target_vars = [ x for x in df.columns if not x in dropped_vars ]
else:
    extra = [ 'sp_1gram', 'sp_2gram', 'sp_3gram', 'sp_4gram',
             'sp_skippy2gram', 'sp_skippy3gram', 'sp_skippy4gram' ]
    dropped_vars.extend(extra)
    target_vars = [ x for x in df.columns if not x in dropped_vars ]    
#
df[target_vars]

,spell,sound,sp_1gram,sp_2gram,sp_3gram,sp_4gram,sp_skippy2gram,sp_skippy3gram,sp_skippy4gram
5669,unwary,ənwɛɹi,"[u, n, w, a, r, y]","[un, nw, wa, ar, ry, u, n, w, a, r, y]","[unw, nwa, war, ary, un, nw, wa, ar, ry, u, n,...","[unwa, nwar, wary, unw, nwa, war, ary, un, nw,...","[unwa, unw…r, unw…y, un…ar, un…a…y, un…ry, u…w...","[unw, un…a, un…r, un…y, u…wa, u…w…r, u…w…y, u…...","[unwa, unw…r, unw…y, un…ar, un…a…y, un…ry, u…w..."
4267,quasi,kwɑsi,"[q, u, a, s, i]","[qu, ua, as, si, q, u, a, s, i]","[qua, uas, asi, qu, ua, as, si, q, u, a, s, i]","[quas, uasi, qua, uas, asi, qu, ua, as, si, q,...","[quas, qua…i, qu…si, q…asi, uasi]","[qua, qu…s, qu…i, q…as, q…a…i, q…si, uas, ua…i...","[quas, qua…i, qu…si, q…asi, uasi, uasi, qua, q..."
353,appointive,əpɔɪntɪv,"[a, p, p, o, i, n, t, i, v, e]","[ap, pp, po, oi, in, nt, ti, iv, ve, a, p, p, ...","[app, ppo, poi, oin, int, nti, tiv, ive, ap, p...","[appo, ppoi, poin, oint, inti, ntiv, tive, app...","[appo, app…i, app…n, app…t, app…i, app…v, app…...","[app, ap…o, ap…i, ap…n, ap…t, ap…i, ap…v, a…po...","[appo, app…i, app…n, app…t, app…i, app…v, ap…o..."
742,bottom,bɑtəm,"[b, o, t, t, o, m]","[bo, ot, tt, to, om, b, o, t, t, o, m]","[bot, ott, tto, tom, bo, ot, tt, to, om, b, o,...","[bott, otto, ttom, bot, ott, tto, tom, bo, ot,...","[bott, bot…o, bot…m, bo…to, bo…t…m, bo…om, b…t...","[bot, bo…t, bo…o, bo…m, b…tt, b…t…o, b…t…m, b…...","[bott, bot…o, bot…m, bo…to, bo…t…m, bo…om, b…t..."
1750,drab,dɹæb,"[d, r, a, b]","[dr, ra, ab, d, r, a, b]","[dra, rab, dr, ra, ab, d, r, a, b]","[drab, dra, rab, dr, ra, ab, d, r, a, b]",[drab],"[dra, dr…b, d…ab, rab, rab, drab]","[drab, dra, dr…b, d…ab, rab, rab, drab]"
...,...,...,...,...,...,...,...,...,...
5101,tangible,tændʒəbəɫ,"[t, a, n, g, i, b, l, e]","[ta, an, ng, gi, ib, bl, le, t, a, n, g, i, b,...","[tan, ang, ngi, gib, ibl, ble, ta, an, ng, gi,...","[tang, angi, ngib, gibl, ible, tan, ang, ngi, ...","[tang, tan…i, tan…b, tan…l, tan…e, ta…gi, ta…g...","[tan, ta…g, ta…i, ta…b, ta…l, ta…e, t…ng, t…n…...","[tang, tan…i, tan…b, tan…l, tan…e, ta…gi, ta…g..."
2623,hushed,həʃt,"[h, u, s, h, e, d]","[hu, us, sh, he, ed, h, u, s, h, e, d]","[hus, ush, she, hed, hu, us, sh, he, ed, h, u,...","[hush, ushe, shed, hus, ush, she, hed, hu, us,...","[hush, hus…e, hus…d, hu…he, hu…h…d, hu…ed, h…s...","[hus, hu…h, hu…e, hu…d, h…sh, h…s…e, h…s…d, h…...","[hush, hus…e, hus…d, hu…he, hu…h…d, hu…ed, h…s..."
4714,showy,ʃoʊi,"[s, h, o, w, y]","[sh, ho, ow, wy, s, h, o, w, y]","[sho, how, owy, sh, ho, ow, wy, s, h, o, w, y]","[show, howy, sho, how, owy, sh, ho, ow, wy, s,...","[show, sho…y, sh…wy, s…owy, howy]","[sho, sh…w, sh…y, s…ow, s…o…y, s…wy, how, ho…y...","[show, sho…y, sh…wy, s…owy, howy, howy, sho, s..."
3173,lipped,ɫɪpt,"[l, i, p, p, e, d]","[li, ip, pp, pe, ed, l, i, p, p, e, d]","[lip, ipp, ppe, ped, li, ip, pp, pe, ed, l, i,...","[lipp, ippe, pped, lip, ipp, ppe, ped, li, ip,...","[lipp, lip…e, lip…d, li…pe, li…p…d, li…ed, l…p...","[lip, li…p, li…e, li…d, l…pp, l…p…e, l…p…d, l…...","[lipp, lip…e, lip…d, li…pe, li…p…d, li…ed, l…p..."


In [116]:
## select data type and define doc_dict
import random
if "sp_" in term_type:
    base_type = "spell"
else:
    base_type = "sound"
doc_dict = { i: x for i, x in enumerate(df[base_type]) }
## check
random.sample(doc_dict.items(), 10)

/var/folders/s2/lk8hdt6j10j0xyycw1lbjsm40000gn/T/ipykernel_44770/1181347170.py:9: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  random.sample(doc_dict.items(), 10)


[(2856, 'fibrous'),
 (557, 'disjointed'),
 (2439, 'conjugal'),
 (3565, 'cooperative'),
 (356, 'off'),
 (3672, 'roughshod'),
 (1361, 'completed'),
 (236, 'meritorious'),
 (3085, 'commanding'),
 (3498, 'considered')]

In [117]:
## select bots for analysis
enable_term_change = False # if you want to change term_type to save time and energy
if enable_term_change:
	term_type = 'sp_skippy4gram'
	print(f"(changed) term_type: {term_type}")

## bot stands for 'bag-of-terms', a generalization of 'bag-of-words'
bots = [ x for x in df[term_type] if len(x) > min_bot_size ] # Crucially
import random
random.sample(bots, 1)

[['outd',
  'out…o',
  'out…o',
  'out…r',
  'ou…do',
  'ou…d…o',
  'ou…d…r',
  'ou…oo',
  'ou…o…r',
  'ou…or',
  'o…tdo',
  'o…td…o',
  'o…td…r',
  'o…t…oo',
  'o…t…o…r',
  'o…t…or',
  'o…doo',
  'o…do…r',
  'o…d…or',
  'o…oor',
  'utdo',
  'utd…o',
  'utd…r',
  'ut…oo',
  'ut…o…r',
  'ut…or',
  'u…doo',
  'u…do…r',
  'u…d…or',
  'u…oor',
  'tdoo',
  'tdo…r',
  'td…or',
  't…oor',
  'door',
  'utdo',
  'utd…o',
  'utd…r',
  'ut…oo',
  'ut…o…r',
  'ut…or',
  'u…doo',
  'u…do…r',
  'u…d…or',
  'u…oor',
  'tdoo',
  'tdo…r',
  'td…or',
  't…oor',
  'door',
  'tdoo',
  'tdo…r',
  'td…or',
  't…oor',
  'door',
  'door',
  'out',
  'ou…d',
  'ou…o',
  'ou…o',
  'ou…r',
  'o…td',
  'o…t…o',
  'o…t…o',
  'o…t…r',
  'o…do',
  'o…d…o',
  'o…d…r',
  'o…oo',
  'o…o…r',
  'o…or',
  'utd',
  'ut…o',
  'ut…o',
  'ut…r',
  'u…do',
  'u…d…o',
  'u…d…r',
  'u…oo',
  'u…o…r',
  'u…or',
  'tdo',
  'td…o',
  'td…r',
  't…oo',
  't…o…r',
  't…or',
  'doo',
  'do…r',
  'd…or',
  'oor',
  'utd',
  'ut…o',
  '

In [118]:
## generate dictionary
from gensim.corpora import Dictionary
diction = Dictionary(bots)
print(diction)
#apply_term_filtering = False
if apply_term_filtering:
    print(f"term filtering applied")
    diction.filter_extremes(no_below = term_minfreq, no_above = abuse_threshold)
else:
    print(f"term filtering not applied")
## check
print(diction)
## generate DTM
corpus = [ diction.doc2bow(bot) for bot in bots if len(bot) > min_bot_size ] # Crucially

Dictionary<196629 unique tokens: ['ary', 'nwa', 'nwar', 'nwa…y', 'nw…r']...>
term filtering applied
Dictionary<96111 unique tokens: ['ary', 'nwa', 'nwar', 'nw…r', 'nw…y']...>


In [119]:
## HDP (n_topics = 15)
import gensim.models
import pyLDAvis.gensim
max_n_topics = 15
hdp15 = gensim.models.HdpModel(corpus, diction, T = max_n_topics, random_state = 1)
vis_data15 = pyLDAvis.gensim.prepare(hdp15, corpus, diction)
pyLDAvis.display(vis_data15)

In [120]:
## save LDAvis output as a html file
save_LDAvis = True
if save_LDAvis:
	lang_dir_name = target_lang_dict[target_lang_key].split()[0]
	vis_output = f"results/LDAvis/{lang_dir_name}/{target_lang_dict[target_lang_key]}{target_class}-HDP-max_ntop{max_n_topics}-{term_type}{accent_status}.html"
	pyLDAvis.save_html(vis_data15, vis_output)

In [121]:
## topic investigation
import numpy as np
import HDP_helper
reload_module = False
if reload_module:
    import importlib
    importlib.reload(HDP_helper)

documents_topics = np.zeros([hdp15.m_T, len(corpus)])
for doc_id, c in enumerate(corpus):
    for topic_id, prob in hdp15[c]:
        documents_topics[topic_id][doc_id] = prob

n_docs_to_show  = 10
n_terms_to_pick = 60
hdp15.optimal_ordering()
for topic_id, probs in enumerate(documents_topics):
    print(f"==============")
    topic_t = hdp15.print_topic(topic_id, topn = n_terms_to_pick)
    print(f"topic_id {topic_id}: {HDP_helper.reformat_topic (topic_t, n_terms_to_pick)}")
    print(f"nonzero count: ", len(probs.nonzero()[0]))
    for doc_id in probs.argsort()[::-1][:n_docs_to_show]:
        doc = doc_dict[doc_id]
        print(f"\t{probs[doc_id]:0.4f}: {doc}")

topic_id 0: 0.001 * ive + 0.001 * t…ve + 0.001 * tive + 0.001 * tiv + 0.001 * ous + 0.001 * ial + 0.001 * ate + 0.001 * tic + 0.0 * t…al + 0.0 * s…ve + 0.0 * n…al + 0.0 * r…al + 0.0 * ant + 0.0 * nti + 0.0 * i…us + 0.0 * r…ti + 0.0 * en…i + 0.0 * si…e + 0.0 * ati + 0.0 * ary + 0.0 * a…ive + 0.0 * i…nt + 0.0 * t…ti + 0.0 * n…nt + 0.0 * r…nt + 0.0 * i…a…e + 0.0 * a…iv + 0.0 * e…e…t + 0.0 * at…e + 0.0 * t…nt + 0.0 * e…ent + 0.0 * e…en + 0.0 * e…nt + 0.0 * e…ive + 0.0 * ious + 0.0 * i…at + 0.0 * ti…l + 0.0 * er…e + 0.0 * a…ve + 0.0 * e…ve + 0.0 * iou + 0.0 * ti…a + 0.0 * i…i…e + 0.0 * ine + 0.0 * ativ + 0.0 * e…i…l + 0.0 * at…ve + 0.0 * ri…l + 0.0 * i…e…t + 0.0 * t…ry + 0.0 * a…al + 0.0 * r…in + 0.0 * tin + 0.0 * ati…e + 0.0 * n…ive + 0.0 * ra…e + 0.0 * sive + 0.0 * e…ia + 0.0 * i…ate + 0.0 * i…en
nonzero count:  1143
	0.9993: continental
	0.9993: contentious
	0.9993: centenarian
	0.9993: secretarial
	0.9993: instructive
	0.9993: constrained
	0.9993: inferential
	0.9993: consonantal
	0.999

In [122]:
## save topic structures
#hdp.get_topics() # =/= show_topics()
#hdp.print_topics()
hdp_topics = hdp15.show_topics(num_topics = max_n_topics,
                               num_words = n_terms_to_show, formatted = False)
hdp_dict = { tid: values for tid, values in hdp_topics }
## convert to Pandas dataframe
topics_df = pd.DataFrame.from_dict(hdp_dict)
#hdp15_topics_out = f"results/terms-by-topics-raw/hdp{max_n_topics}_topics_raw.csv"
hdp15_topics_out = f"results/terms-by-topics-raw/{lang_dir_name}/{target_lang_dict[target_lang_key]}{target_class}-topics{max_n_topics}-{term_type}{accent_status}.csv"
topics_df.to_csv(hdp15_topics_out, header = False, index = None)

In [123]:
## HDP (n_topics = 45)
import gensim.models
import pyLDAvis.gensim
max_n_topics = 45
hdp45 = gensim.models.HdpModel(corpus, diction, T = max_n_topics, random_state = 1)
vis_data45 = pyLDAvis.gensim.prepare(hdp45, corpus, diction)
pyLDAvis.display(vis_data45)

In [124]:
## save LDAvis output as a html file
lang_dir_name = target_lang_dict[target_lang_key].split()[0]
save_LDAvis = True
if save_LDAvis:
	vis_output = f"results/LDAvis/{lang_dir_name}/{target_lang_dict[target_lang_key]}{target_class}-HDP-max_ntop{max_n_topics}-{term_type}{accent_status}.html"
	pyLDAvis.save_html(vis_data45, vis_output)

In [125]:
## save topic structures
#hdp.get_topics() # =/= show_topics()
#hdp.print_topics()
hdp_topics = hdp45.show_topics(num_topics = max_n_topics,
                               num_words = n_terms_to_show, formatted = False)
hdp_dict = { tid: values for tid, values in hdp_topics }
## convert to Pandas dataframe
topics_df = pd.DataFrame.from_dict(hdp_dict)
#hdp45_topics_out = f"results/terms-by-topics-raw/hdp{max_n_topics}_topics_raw.csv"
hdp45_topics_out = f"results/terms-by-topics-raw/{lang_dir_name}/{target_lang_dict[target_lang_key]}{target_class}-topics{max_n_topics}-{term_type}{accent_status}.csv"
topics_df.to_csv(hdp45_topics_out, header = False, index = None)

In [126]:
## topic investigation
import numpy as np
import HDP_helper
documents_topics = np.zeros([hdp45.m_T, len(corpus)])
for doc_id, c in enumerate(corpus):
    for topic_id, prob in hdp45[c]:
        documents_topics[topic_id][doc_id] = prob
#
n_docs_to_show  = 10
n_terms_to_pick = 10
hdp45.optimal_ordering()
for topic_id, probs in enumerate(documents_topics):
    print(f"==============")
    topic_t = hdp45.print_topic(topic_id, topn = n_terms_to_pick)
    print(f"topic_id {topic_id}: {HDP_helper.reformat_topic (topic_t, n_terms_to_pick)}")
    print(f"nonzero count: {len(probs.nonzero()[0])}")
    for doc_id in probs.argsort()[::-1][:n_docs_to_show]:
        doc = doc_dict[doc_id]
        print(f"\t{probs[doc_id]:0.4f}: {doc}")

topic_id 0: 0.001 * a…ing + 0.001 * e…ing + 0.001 * t…ng + 0.001 * e…in + 0.001 * tin + 0.001 * a…i…g + 0.001 * e…i…g + 0.001 * a…ng + 0.001 * ous + 0.001 * e…ng
nonzero count: 914
	0.9993: interesting
	0.9993: consonantal
	0.9993: interracial
	0.9993: contrasting
	0.9993: distressing
	0.9992: residential
	0.9992: substantial
	0.9992: indifferent
	0.9992: stimulating
	0.9992: nonstandard
topic_id 1: 0.002 * ate + 0.001 * r…te + 0.001 * ra…e + 0.001 * ral + 0.001 * rat + 0.001 * era + 0.001 * ated + 0.001 * rate + 0.001 * e…at + 0.001 * e…te
nonzero count: 431
	0.9992: intemperate
	0.9992: sedimentary
	0.9992: amalgamated
	0.9992: utilitarian
	0.9992: rudimentary
	0.9992: unpublished
	0.9992: unimportant
	0.9992: sentimental
	0.9991: sacramental
	0.9991: intimidated
topic_id 2: 0.001 * ate + 0.001 * e…te + 0.001 * f…ed + 0.001 * e…ted + 0.001 * at…d + 0.001 * ated + 0.001 * e…t…d + 0.001 * ied + 0.001 * fi…d + 0.001 * i…ted
nonzero count: 422
	0.9992: unconcerned
	0.9992: uncollected
	0

In [127]:
## HDP (n_topics = 90)
import gensim.models
import pyLDAvis.gensim
max_n_topics = 90
hdp90 = gensim.models.HdpModel(corpus, diction, T = max_n_topics,
                               random_state = 1,
                               #var_converge = 0.001
                               )
vis_data90 = pyLDAvis.gensim.prepare(hdp90, corpus, diction)
pyLDAvis.display(vis_data90)

In [128]:
## save LDAvis output as a html file
save_LDAvis = True
if save_LDAvis:
	lang_dir_name = target_lang_dict[target_lang_key].split()[0]
	vis_output = f"results/LDAvis/{lang_dir_name}/{target_lang_dict[target_lang_key]}{target_class}-HDP-max_ntop{max_n_topics}-{term_type}{accent_status}.html"
	pyLDAvis.save_html(vis_data90, vis_output)

In [129]:
## save topic structures
hdp_topics = hdp90.show_topics(num_topics = max_n_topics,
                               num_words = n_terms_to_show, formatted = False)
hdp_dict = { tid: values for tid, values in hdp_topics }
## convert to Pandas dataframe
topics_df = pd.DataFrame.from_dict(hdp_dict)
#hdp90_topics_out = f"results/terms-by-topics-raw/hdp{max_n_topics}_topics_raw.csv"
hdp90_topics_out = f"results/terms-by-topics-raw/{lang_dir_name}/{target_lang_dict[target_lang_key]}{target_class}-topics{max_n_topics}-{term_type}{accent_status}.csv"
topics_df.to_csv(hdp90_topics_out, header = False, index = None)

In [130]:
## topic investigation
import numpy as np
import HDP_helper
documents_topics = np.zeros([hdp90.m_T, len(corpus)])
for doc_id, c in enumerate(corpus):
    for topic_id, prob in hdp90[c]:
        documents_topics[topic_id][doc_id] = prob
## investigate topics
n_docs_to_show  = 10
n_terms_to_pick = 10
hdp90.optimal_ordering()
for topic_id, probs in enumerate(documents_topics):
    print(f"==============")
    #topic_encoding = ", ".join(hdp.show_topic(topic_id))
    topic_t = hdp90.print_topic(topic_id, topn = n_terms_to_pick)
    print(f"topic_id {topic_id}: {HDP_helper.reformat_topic (topic_t, n_terms_to_pick)}")
    print(f"nonzero count: ", len(probs.nonzero()[0]))
    for doc_id in probs.argsort()[::-1][:n_docs_to_show]:
        doc = doc_dict[doc_id]
        print(f"\t{probs[doc_id]:0.4f}: {doc}")

topic_id 0: 0.003 * able + 0.003 * abl + 0.002 * i…le + 0.001 * e…ble + 0.001 * i…ble + 0.001 * r…ble + 0.001 * e…bl + 0.001 * e…abl + 0.001 * t…ble + 0.001 * i…bl
nonzero count:  604
	0.9992: intractable
	0.9992: untraceable
	0.9992: interracial
	0.9992: retractable
	0.9992: undesirable
	0.9992: predictable
	0.9992: permissible
	0.9992: convertible
	0.9992: inheritable
	0.9992: irreparable
topic_id 1: 0.001 * ive + 0.001 * n…ng + 0.001 * a…ing + 0.001 * n…ing + 0.001 * n…in + 0.001 * e…ing + 0.001 * e…in + 0.001 * e…en + 0.001 * n…i…g + 0.001 * a…ng
nonzero count:  576
	0.9992: substantial
	0.9992: indifferent
	0.9992: independent
	0.9992: nonstandard
	0.9991: complaining
	0.9991: inoffensive
	0.9991: commonsense
	0.9991: outstanding
	0.9991: encouraging
	0.9990: commonplace
topic_id 2: 0.003 * ate + 0.002 * e…te + 0.002 * ated + 0.002 * r…te + 0.002 * at…d + 0.002 * l…te + 0.001 * e…ted + 0.001 * ra…e + 0.001 * u…ate + 0.001 * e…ate
nonzero count:  314
	0.9992: incompetent
	0.9992: i